# Metrics Notebook

## Imports & Setup

In [20]:
import json
import time
from typing import List, Dict, Any
import pandas as pd

from llama_index.core import (
    StorageContext,
    load_index_from_storage
)

from engine import MagicJudgeEngine 


## Query Function

In [17]:


def ask_judge(
    judge, 
    question: str,
    history: List[Dict] = None,
    collect_tokens: bool = False
) -> Dict[str, Any]:
    """
    Realiza una consulta al juez. 
    Robusta a fallos: Si explota, devuelve un diccionario con el error 
    en lugar de detener todo el script de evaluación.
    """
    if history is None:
        history = []

    t_start = time.time()
    
    # Variables de control
    full_response = ""
    tokens = []
    error_msg = None
    success = False

    try:
        # Ejecutamos la query
        stream = judge.query(question, history=history)

        # Consumimos el stream
        for token in stream:
            # Manejo defensivo por si LlamaIndex cambia la estructura del objeto
            delta = getattr(token, "delta", str(token))
            
            if delta:
                full_response += delta
                if collect_tokens:
                    tokens.append(delta)
        
        success = True

    except Exception as e:
        # Capturamos el error para que el loop de 100 preguntas no se detenga
        error_msg = str(e)
        print(f"⚠️ Error procesando pregunta: {question[:30]}... | {error_msg}")

    latency = time.time() - t_start

    return {
        "question": question,
        "generated_answer": full_response.strip(), # Limpiamos espacios
        "ground_truth": None, # Esto lo llenarás tú después al cruzar con tu dataset
        "latency": latency,
        "success": success,
        "error": error_msg,
        "tokens": tokens if collect_tokens else None
    }

In [6]:
judge = MagicJudgeEngine()

[LOG] Loading Rules Index...
[LOG] Loading Cards Index...


In [18]:
# test with one question
question = "If I attack with a creature with Deathtouch and Trample and it gets blocked, how much damage do I need to assign to the blocker?"
response = ask_judge(judge,question)

[LOG] Search Query: If I attack with a creature with Deathtouch and Trample and it gets blocked, how much damage do I need to assign to the blocker?
[LOG] No exact cards found. Running semantic search.

[LOG] RETRIEVAL CANDIDATES (After Filtering)
 - [Rule] 702.19b                        (sc: 0.70)
 - [Rule] 702.19d                        (sc: 0.64)
 - [Rule] 510.1c                         (sc: 0.63)
 - [Rule] 702.19e                        (sc: 0.61)
 - [Rule] 510.1d                         (sc: 0.60)
 - [Rule] 702.2c                         (sc: 0.59)
 - [Card] Enlarge                        (sc: 0.59)
 - [Rule] 510.1a                         (sc: 0.58)
 - [Rule] 510.1                          (sc: 0.58)
 - [Card] Ride Down                      (sc: 0.57)
 - [Card] Mirror Shield                  (sc: 0.55)
 - [Card] Deathcoil Wurm                 (sc: 0.54)
 - [Card] Fight to the Death             (sc: 0.54)



In [19]:
response

{'question': 'If I attack with a creature with Deathtouch and Trample and it gets blocked, how much damage do I need to assign to the blocker?',
 'generated_answer': 'Ah, the intricacies of combat mechanics! Let us delve into the nuances of trample and deathtouch.\n\n### 1. The Interaction\nYou are attacking with a creature that possesses both deathtouch and trample, and it has been blocked by an opposing creature. The key here is understanding how damage assignment works in this scenario.\n\n### 2. The Logic (Step-by-Step)\n- **Deathtouch Mechanic**: A creature with deathtouch only needs to assign a single point of damage to a blocking creature for that damage to be considered lethal. This is crucial because it affects how much damage you must assign.\n- **Trample Mechanic**: When a creature with trample is blocked, it must assign enough damage to the blocking creature to satisfy the lethal damage requirement before it can assign any excess damage to the defending player or planeswalk

## Dataset Questions and answers

In [26]:
# 1. Cargar el archivo JSON
with open('questions_v1.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 2. Crear el DataFrame de Definiciones (Metadata)
df_bloques = pd.DataFrame(data['blocks_definition'])

# 3. Crear el DataFrame de Preguntas
df_questions = pd.DataFrame(data['questions'])

In [41]:
df_bloques

,block,theme,focus
0,1,Combat & Basic Keywords,Fundamentals of damage and common keywords.
1,2,The Stack & Spell Interaction,"LIFO order, counterspells, and independence of..."
2,3,Targets & Triggers,"Legal targets, ward, and ETB/LTB triggers."
3,4,Resources & Mana,"Mana pool, land types, and cost taxes."
4,5,State-Based Actions (SBA),"Legend rule, 0 toughness, and game loss condit..."
5,6,Replacement Effects & Control,Instead effects and continuous control changes.
6,7,Commander & Multiplayer,"Color identity, commander tax, and command zon..."
7,8,Copies & Transformations,"Copyable values, morph, and double-faced cards."
8,9,Costs & X Spells,Mana value on stack vs other zones and cost ca...
9,10,Elite Interactions (Pro Level),"Layers, complex dependencies, and timestamp lo..."


In [42]:
df_questions['block'] = (df_questions['id'] - 1) // 10 + 1
df_questions = df_questions.merge(df_bloques[['block', 'theme']], on='block')

In [44]:
df_questions.head(20)

,id,difficulty,category,question,ground_truth,block,theme
0,1,Easy,Keywords,If I attack with a creature with [[Lifelink]] ...,Yes. Lifelink causes you to gain life equal to...,1,Combat & Basic Keywords
1,2,Easy,Keywords,Can a creature with [[Flying]] be blocked by a...,Yes. Reach is a keyword specifically designed ...,1,Combat & Basic Keywords
2,3,Easy,Keywords,"If my creature has [[Vigilance]], does it stil...",Yes. Vigilance means the creature does not tap...,1,Combat & Basic Keywords
3,4,Easy,Keywords,Does [[Ward 2]] counter a spell if the opponen...,Yes. Ward is a triggered ability. When the cre...,1,Combat & Basic Keywords
4,5,Medium,Keywords,If I attack with a creature with [[Deathtouch]...,You only need to assign 1 point of damage to t...,1,Combat & Basic Keywords
5,6,Easy,Keywords,Can a creature with [[Haste]] activate an abil...,Yes. Haste removes the restriction that preven...,1,Combat & Basic Keywords
6,7,Medium,Turn Structure,Can I cast a creature spell during my opponent...,"No. Without Flash, creatures can only be cast ...",1,Combat & Basic Keywords
7,8,Easy,Keywords,If my creature has [[First Strike]] and my opp...,No. Your creature deals damage in the first st...,1,Combat & Basic Keywords
8,9,Medium,Keywords,"If a creature has [[Indestructible]], does it ...",Yes. Indestructible only prevents destruction ...,1,Combat & Basic Keywords
9,10,Easy,Keywords,"If I have [[Double Strike]], do I get two trig...",Yes. The creature deals damage in two separate...,1,Combat & Basic Keywords


## Metric Evaluation

In [ ]:
# pending